# **🧠 Chatbot de Aconselhamento para Saúde Mental**

### **🌐 Curso: Redes Neurais e PLN**

#### **Objetivo:** Neste projeto, foi construído um chatbot treinado com conversas reais de aconselhamento psicológico, utilizando o modelo `DialoGPT` e o dataset `mental_health_counseling_conversations` da Hugging Face. O objetivo é permitir que o modelo interaja em linguagem natural e ofereça respostas empáticas em contextos de saúde mental.

> Versão: Maio/2025


## **🔧 Instalação de bibliotecas**
Nesta etapa é instalado as bibliotecas necessárias para:

- Carregar datasets (com `datasets`)
- Utilizar modelos e tokenizadores da Hugging Face (com `transformers`)
- Criar interfaces interativas (com `gradio`)
- Treinar modelos com GPU (com `torch`)

In [ ]:
!pip install -r requirements.txt

In [ ]:
# No Colab
!pip install datasets gradio

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import gradio as gr
import torch

## **🗃️ 1. Carregar o dataset**

Foi utilizado o dataset `mental_health_counseling_conversations`, que contém pares de diálogos simulando atendimentos psicológicos. Cada amostra possui:

- `Context`: a fala do usuário (cliente)
- `Response`: a resposta do conselheiro ou psicólogo

Esses dados servirão de base para o treinamento do modelo de linguagem.

In [ ]:
dataset = load_dataset("Amod/mental_health_counseling_conversations")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

## **🔎 2. Pré-processar os dados**

Foi ajustado os dados para um formato mais adequado para treinamento. Cada item passa a ter:

- `input`: texto de entrada (Context)
- `output`: texto-alvo para geração (Response)

Essa estrutura facilita a tokenização e o treino supervisionado do modelo.

In [ ]:
def format_conversation(example):
    return {
        "input": example["Context"],
        "output": example["Response"]
    }

formatted_data = dataset["train"].map(format_conversation)

formatted_data = dataset["train"].map(format_conversation)
formatted_data = formatted_data.filter(lambda x: x is not None)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3512 [00:00<?, ? examples/s]

## **🪙 3. Tokenização**
Aqui preparo os dados para o modelo `DialoGPT`. As etapas são:

- Definir um token de padding (o modelo não tem `pad_token` por padrão)
- Concatenar entrada e saída em uma única sequência
- Criar os `labels` com `-100` nos tokens da entrada para que o modelo aprenda apenas a prever a saída

A tokenização respeita o limite de 512 tokens, que é o máximo aceito pelo modelo small, já o medium aceita mais, porém por conta de eficiência foi escolhido 512.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Definir pad_token como o eos_token (necessário para evitar erro)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    input_texts = examples["input"]
    output_texts = examples["output"]

    input_ids_list = []
    labels_list = []
    attention_masks_list = []

    for input_text, output_text in zip(input_texts, output_texts):
        # Garantir que são strings
        input_text = str(input_text)
        output_text = str(output_text)

        # Concatenar entrada + resposta
        full_text = input_text + tokenizer.eos_token + output_text + tokenizer.eos_token

        # Tokenizar tudo de uma vez
        tokenized = tokenizer(
            full_text,
            truncation=True,
            padding="max_length",
            max_length=512
        )

        # Criar os labels ignorando a parte da pergunta (com -100)
        input_len = len(tokenizer(input_text + tokenizer.eos_token)["input_ids"])
        full_labels = [-100] * input_len + tokenized["input_ids"][input_len:]
        full_labels = full_labels[:512]  # garantir mesmo tamanho

        # Criar attention_mask manualmente
        attention_mask = [1 if token != tokenizer.pad_token_id else 0 for token in tokenized["input_ids"]]

        input_ids_list.append(tokenized["input_ids"])
        labels_list.append(full_labels)
        attention_masks_list.append(attention_mask)

    return {
        "input_ids": input_ids_list,
        "labels": labels_list,
        "attention_mask": attention_masks_list
    }

tokenized_dataset = formatted_data.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

## **😊 4. Carregar o modelo**

Nesta célula, foi carregado o modelo `microsoft/DialoGPT-medium` da Hugging Face. Ele é um modelo pré-treinado em diálogos do Reddit, e será ajustado (fine-tuned) com o dataset de saúde mental.

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## **🎓 5. Treinar o modelo** - Fine Tuning

Foi definido as configurações do processo de treino com o `TrainingArguments`:

- Diretório de saída dos resultados
- Tamanho do batch por dispositivo
- Número de épocas (fine-tuning leve)
- Quantidade de logs

Iniciei o processo de fine-tuning com base nos exemplos do dataset. Este passo ajusta os pesos do modelo `DialoGPT` para que ele aprenda o estilo de linguagem do domínio de saúde mental.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="no"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: heitor-sabino (heitor-sabino-ufla-universidade-federal-de-lavras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.077900
20,2.365300
30,2.706200
40,2.269900
50,2.489800
60,1.814500
70,1.841100
80,2.158200
90,1.925200
100,2.192500


TrainOutput(global_step=5268, training_loss=1.200987002029506, metrics={'train_runtime': 4807.2894, 'train_samples_per_second': 2.192, 'train_steps_per_second': 1.096, 'total_flos': 9784790517547008.0, 'train_loss': 1.200987002029506, 'epoch': 3.0})

- **Aqui salva o modelo para ser usado posteriormente sem necessidade de "fine-tunar" novamente (Uma etapa demorada)**

In [ ]:
model.save_pretrained("chatbot-model")
tokenizer.save_pretrained("chatbot-model")

('chatbot-model/tokenizer_config.json',
 'chatbot-model/special_tokens_map.json',
 'chatbot-model/vocab.json',
 'chatbot-model/merges.txt',
 'chatbot-model/added_tokens.json',
 'chatbot-model/tokenizer.json')

### **💪 5.1 Após modelo treinado**
- *Rode essa linha para testar o chatbot e não passar pela espera de treinamento (fine tuning)*

In [1]:
!unzip chatbot-model.zip -d /content/chatbot-model

Archive:  chatbot-model.zip
  inflating: /content/chatbot-model/config.json  
  inflating: /content/chatbot-model/generation_config.json  
  inflating: /content/chatbot-model/tokenizer_config.json  
  inflating: /content/chatbot-model/model.safetensors  
  inflating: /content/chatbot-model/merges.txt  
  inflating: /content/chatbot-model/vocab.json  
  inflating: /content/chatbot-model/special_tokens_map.json  
  inflating: /content/chatbot-model/tokenizer.json  


In [5]:
model = AutoModelForCausalLM.from_pretrained("chatbot-model")
tokenizer = AutoTokenizer.from_pretrained("chatbot-model")

## **🗣️ 6. Função de geração de resposta**

Definimos a função `conversar()` que:

- Recebe um texto do usuário (`prompt`)
- Constrói a sequência de entrada para o modelo
- Gera uma resposta com `model.generate()`
- Decodifica os tokens para texto legível

Ela será usada para testar o chatbot treinado.

In [ ]:
def conversar(prompt, model, tokenizer):
    device = next(model.parameters()).device

    inputs = tokenizer(prompt + tokenizer.eos_token, return_tensors="pt").to(device)

    outputs = model.generate(
    **inputs,
    max_length=200,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.2
  )

    resposta = tokenizer.decode(outputs[:, inputs["input_ids"].shape[-1]:][0], skip_special_tokens=True)
    return resposta

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

## **🤖 7. Interface via terminal**

É criado um loop interativo usando `input()` para permitir que o usuário converse com o chatbot diretamente no terminal. A conversa segue até que a palavra `sair` seja digitada.

In [ ]:
print("Chatbot de Saúde Mental (digite 'sair' para encerrar)\n")

while True:
    pergunta = input("Você: ")
    if pergunta.lower() in ["sair", "exit", "quit"]:
        print("Encerrando a conversa.")
        break

    resposta = conversar(pergunta, model, tokenizer)
    print("Bot:", resposta)

Chatbot de Saúde Mental (digite 'sair' para encerrar)

Você: I'm not feeling well mentally. Lately, I've been overwhelmed with emotions, and it's hard to focus or find motivation. 
Bot: Sometimes when we are overwhelmed by our emotions, we don't pay any attention to our thoughts or feelings and instead focus on the sensations that are actually happening in our body. This could be either your mind shutting off your thoughts or your body shutting off your emotions. If you can find a way to get back into your body, this is possible.
Você: sair
Encerrando a conversa.


## **🖼️ 7.1 Interface via Gradio** - (indo além!)

Este código cria uma interface de chatbot com a biblioteca Gradio, que facilita a criação de interfaces web para funções em Python.

In [ ]:
def responder_gradio(prompt):
    return conversar(prompt, model, tokenizer)

interface = gr.Interface(
    fn=responder_gradio,
    inputs=gr.Textbox(lines=5, placeholder="Describe how you're feeling..."),
    outputs="text",
    title="Mental Health Chatbot",
    description="This chatbot was fine-tuned using real mental health counseling conversations. Feel free to share your thoughts. 🤖🧠"
)

interface.launch(share=True)